In [11]:
import os
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

def batch_format_and_save_images(input_folder, output_folder, batch_size=64, target_size=224):
    os.makedirs(output_folder, exist_ok=True)
    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    total = len(image_files)
    for i in range(0, total, batch_size):
        batch_files = image_files[i:i+batch_size]
        for fname in batch_files:
            in_path = os.path.join(input_folder, fname)
            out_path = os.path.join(output_folder, fname)
            try:
                arr = load_and_pad_square(in_path, target_size=target_size)
                img = array_to_img(arr)
                img.save(out_path)
            except Exception as e:
                print(f"Failed to process {fname}: {e}")
        print(f"Processed {min(i+batch_size, total)}/{total} images.")

# Example usage:
# batch_format_and_save_images("images-1", "images-1-formatted", batch_size=64, target_size=224)

In [12]:
batch_format_and_save_images("images-1", "images-1-formatted", batch_size=64, target_size=224) 
batch_format_and_save_images("images-2", "images-2-formatted", batch_size=64, target_size=224) 
batch_format_and_save_images("images-3", "images-3-formatted", batch_size=64, target_size=224)
batch_format_and_save_images("images-4", "images-4-formatted", batch_size=64, target_size=224)
batch_format_and_save_images("images-5", "images-5-formatted", batch_size=64, target_size=224)
batch_format_and_save_images("images-6", "images-6-formatted", batch_size=64, target_size=224)
batch_format_and_save_images("images-7", "images-7-formatted", batch_size=64, target_size=224)
batch_format_and_save_images("images-8", "images-8-formatted", batch_size=64, target_size=224)
batch_format_and_save_images("images-9", "images-9-formatted", batch_size=64, target_size=224)
batch_format_and_save_images("images-10", "images-10-formatted", batch_size=64, target_size=224)

C:\Users\Malthe\AppData\Local\Temp\ipykernel_13628\2837782236.py:20: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize(new_size, Image.LANCZOS)


Processed 64/5000 images.
Processed 128/5000 images.
Processed 192/5000 images.
Processed 256/5000 images.
Processed 320/5000 images.
Processed 384/5000 images.
Processed 448/5000 images.
Processed 512/5000 images.
Processed 576/5000 images.
Processed 640/5000 images.
Processed 704/5000 images.
Processed 768/5000 images.
Processed 832/5000 images.
Processed 896/5000 images.
Processed 960/5000 images.
Processed 1024/5000 images.
Processed 1088/5000 images.
Processed 1152/5000 images.
Processed 1216/5000 images.
Processed 1280/5000 images.
Processed 1344/5000 images.
Processed 1408/5000 images.
Processed 1472/5000 images.
Processed 1536/5000 images.
Processed 1600/5000 images.
Processed 1664/5000 images.
Processed 1728/5000 images.
Processed 1792/5000 images.
Processed 1856/5000 images.
Processed 1920/5000 images.
Processed 1984/5000 images.
Processed 2048/5000 images.
Processed 2112/5000 images.
Processed 2176/5000 images.
Processed 2240/5000 images.
Processed 2304/5000 images.
Processe

In [3]:
import pandas as pd

# Load the CSV file with the format of "train-ds-ft.csv"
df = pd.read_csv("training-ds-ft.csv")
print(df.head())

                                       id;angle;file
0  6886d4d11c95c1200fdeebd5;-3.271121386604701;68...
1  6888527baf0ce92103121164;-2.700237618768623;68...
2  68875f2a1c95c1200fdef81d;-7.312921806443301;68...
3  68877c8b1c95c1200fdefd45;0.9269297042023048;68...
4  688756641c95c1200fdef682;2.1856050617065783;68...


In [13]:
import os
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import load_img, img_to_array
# Build regression model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
from PIL import Image

def load_and_pad_square(img_path, target_size=224):
    # Load image
    img = Image.open(img_path).convert('RGB')
    # Compute scale and new size
    ratio = float(target_size) / max(img.size)
    new_size = tuple([int(x * ratio) for x in img.size])
    img = img.resize(new_size, Image.LANCZOS)
    # Create new black square image and paste resized image onto center
    new_img = Image.new("RGB", (target_size, target_size))
    paste_pos = ((target_size - new_size[0]) // 2, (target_size - new_size[1]) // 2)
    new_img.paste(img, paste_pos)
    # Convert to array
    arr = img_to_array(new_img)
    return arr
# Update input shape and image loading
IMG_HEIGHT = 224
IMG_WIDTH = 224

# Parse the CSV to split columns
df_split = df['id;angle;file'].str.split(';', expand=True)
df_split.columns = ['id', 'angle', 'file']

# List all image files in the directory
image_dir = "images-1-formatted"
all_image_files = set(os.listdir(image_dir))

# Prepare matched image paths and labels
matched_image_paths = []
matched_angles = []

for fname in all_image_files:
    row = df_split[df_split['file'] == fname]
    if not row.empty:
        matched_image_paths.append(f"{image_dir}/{fname}")
        matched_angles.append(float(row.iloc[0]['angle']))

angles = np.array(matched_angles)
# Load the formatted images
X = []
for img_path in matched_image_paths:
    img = load_img(img_path, target_size=(64, 64))
    img_arr = img_to_array(img)
    X.append(img_arr)
X = np.array(X) / 255.0
# Load images and preprocess
#X = []
#for img_path in matched_image_paths:
 #   img_arr = load_and_pad_square(img_path, target_size=224)
  #  X.append(img_arr)
#X = np.array(X, dtype=np.float32) / 255.0

# Custom reward metric
def within_margin(y_true, y_pred):
    return K.mean(K.cast(K.less_equal(K.abs(y_true - y_pred), 1), K.floatx()))



In [17]:
other_image_dir = "images-2"
other_image_files = set(os.listdir(other_image_dir))

# Find matching images in the new folder using df_split
other_matched_image_paths = []
other_matched_angles = []

for idx, row in df_split.iterrows():
    fname = row['file']
    if fname in other_image_files:
        other_matched_image_paths.append(f"{other_image_dir}/{fname}")
        other_matched_angles.append(float(row['angle']))

if other_matched_image_paths:
    other_angles = np.array(other_matched_angles)

    # Load and preprocess images
    other_X = []
    for img_path in other_matched_image_paths:
        img_arr = load_and_pad_square(img_path, target_size=(224))
        other_X.append(img_arr)
    other_X = np.array(other_X) / 255.0

C:\Users\Malthe\AppData\Local\Temp\ipykernel_13628\1711812072.py:20: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize(new_size, Image.LANCZOS)


In [ ]:
import numpy as np
from tensorflow.keras import layers, models, optimizers, callbacks, backend as K

# angles in degrees, convert to radians
angles_rad = np.deg2rad(angles)
targets = np.stack([np.cos(angles_rad), np.sin(angles_rad)], axis=1)
def build_custom_cnn(input_shape=(224, 224, 3)):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(2, activation='linear')  # Output 2 values: cos and sin
    ])
    return model
model = build_custom_cnn(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    X, targets,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop]
)
other_angles_rad = np.deg2rad(other_angles)
other_targets = np.stack([np.cos(other_angles_rad), np.sin(other_angles_rad)], axis=1)
loss, mse = model.evaluate(other_X, other_targets, verbose=2)
print(f"Performance on {other_image_dir}: Loss={loss:.4f}, MSE={mse:.4f}")
model.save_weights("cossin.h5")
import numpy as np

# Predict cos/sin for the evaluation set in batches to avoid MemoryError
preds = model.predict(other_X, batch_size=32)

# Convert predictions and targets to angles in degrees
pred_angles_rad = np.arctan2(preds[:, 1], preds[:, 0])
pred_angles_deg = np.rad2deg(pred_angles_rad)

true_angles_deg = other_angles  # Assuming other_angles is in degrees

# Custom evaluation metric: percentage within margin (e.g., 0.8 degrees)
margin = 1
angle_diff = np.abs(pred_angles_deg - true_angles_deg)
# Handle wrap-around at 180/-180
angle_diff = np.minimum(angle_diff, 360 - angle_diff)
within_margin = np.mean(angle_diff <= margin)

print(f"Custom evaluation: {within_margin*100:.2f}% predictions within {margin} degrees of ground truth")

ValueError: in user code:

    File "c:\Users\Malthe\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Malthe\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Malthe\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Malthe\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Malthe\anaconda3\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Malthe\anaconda3\lib\site-packages\keras\src\engine\input_spec.py", line 298, in assert_input_compatibility
        raise ValueEr